# Finalize


In [1]:
### EXTRACT KEYWORDS FROM IMAGE ###

from smartbard.image_model.image_keywords import image_to_keywords, select_keyword

#~~~~~~~~~ temp path ~~~~~~~~~~~~
path = '/Users/Marco/code/rcfeord/smartbard/wip/5490264.jpg'
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
image_keywords_df = image_to_keywords(path, 5)
image_keywords_df


,Id,Label,Probability
0,n03109150,corkscrew,0.149547
1,n07248320,book_jacket,0.107495
2,n07892512,red_wine,0.042941
3,n07768694,pomegranate,0.026097
4,n12620546,hip,0.023215


In [2]:
### SELECT BEST KEYWORDS ###

keywords_list = select_keyword(image_keywords_df, 1)
keywords_list


['corkscrew']

In [3]:
### DEFINE STARTING SEQUENCE FOR GPT2

start_sequence = f"<LS> <KS> {' '.join(keywords_list)} <KE>"
start_sequence


'<LS> <KS> corkscrew <KE>'

In [4]:
### GENERATE TEXT WITH GPT2

# generated_limerick = generated_text(sequence)

#~~~~~~~ fake generation ~~~~~~~~
import pandas as pd
data_folder = '../raw_data/'
data_file = 'limerick_dataset_oedilf_v3.json'
file = os.path.join(data_folder, data_file)
df_raw = pd.read_json(file)
df = df_raw[df_raw['is_limerick'] == True]

from smartbard.data_processing.extract_keywords import df_add_noun_keyword

df_kwrd = df_add_noun_keyword(df.iloc[:10])

from smartbard.data_processing.gpt2_formats import limerick_add_special_tkns

df_tkns = limerick_add_special_tkns(df_kwrd)

limerick_tkns = df_tkns['limerick_tk_added'].iloc[0]
limerick_tkns
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


[nltk_data] Downloading package punkt to /Users/Marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Marco/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


"<LS> <KS> bovine <KE> If a bovine in some leafy glade <L0> Should swallow a live hand-grenade <L1> Then surely it's vile <L2> (Though the word makes me smile); <L3> It's abominable — that's what he's made. <L4> <LE> "

In [5]:
### MASK WORDS ###

from smartbard.data_processing.bert_preproc import zorro

masked_limerick = zorro(limerick_tkns)[0]
masked_limerick


"<LS> <KS> bovine <KE> If a bovine in some leafy [MASK] <L0> Should swallow a live hand-[MASK] <L1> Then surely it's [MASK] <L2> (Though the word makes me [MASK]); <L3> It's abominable — that's what he's [MASK]. <L4> <LE> "

In [6]:
### CLEAN TEXT FOR BERT ###

from smartbard.data_processing.bert_preproc import limerick_cleaner

masked_limerick_clean = limerick_cleaner(masked_limerick)
print(masked_limerick_clean)


If a bovine in some leafy [MASK] 
Should swallow a live hand-[MASK] 
Then surely it's [MASK] 
(Though the word makes me [MASK]); 
It's abominable — that's what he's [MASK].


In [7]:
### PREDICT WORDS WITH BERT ###

from smartbard.bert_model.bert_predict_masked_words import rhyme_word_pred
bert_words = rhyme_word_pred(masked_limerick_clean)
bert_words


/Users/Marco/.pyenv/versions/3.10.6/envs/smartbard/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


([('country', 0.09774240106344223),
  ('world', 0.04276498407125473),
  ('place', 0.024797478690743446),
  ('south', 0.013667088933289051),
  ('city', 0.013608594425022602),
  ('part', 0.00873620342463255),
  ('corner', 0.008076431229710579),
  ('earth', 0.007418944966048002),
  ('land', 0.007037917152047157),
  ('way', 0.006921724881976843),
  ('places', 0.006864884868264198),
  ('that', 0.005984313320368528),
  ('spring', 0.005882159806787968),
  ('north', 0.005221132189035416),
  ('one', 0.005194451659917831),
  ('there', 0.004820651840418577),
  ('somewhere', 0.004697255790233612),
  ('continent', 0.004646145273000002),
  ('it', 0.004198145121335983),
  ('town', 0.004193942528218031),
  ('tree', 0.004188721068203449),
  ('sea', 0.004157190211117268),
  ('west', 0.003955739550292492),
  ('all', 0.00395383033901453),
  (',', 0.003920763731002808),
  ('mountain', 0.0038950503803789616),
  ('field', 0.0038088886067271233),
  ('forest', 0.0037242593243718147),
  ('some', 0.0037160292267

In [8]:
### PICK BEST RHYMES AND PRODUCE FINAL LIMERICK ###

from smartbard.data_processing.rhymes import make_rhymes

final_limerick = make_rhymes(masked_limerick_clean, bert_words)
final_limerick

"If a bovine in some leafy land \nShould swallow a live hand-and \nThen surely it's real \n(Though the word makes me feel); \nIt's abominable — that's what he's and."